In [20]:
def read_list(text_path):
    lsit=[]
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            lsit.append(line_list.replace('\n',''))
    return lsit
train_x=read_list('../data/pre_data.txt')
print(train_x[0])

穿着背心的*醒来，看看手机，三点了。


In [39]:
#ban是cls的后缀
ban='_1637318143.8513994_2_0.7074_0.7111'


In [22]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F


class fn_cls(nn.Module):
    def __init__(self,device):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-chinese")
        self.model.resize_token_embeddings(len(tokenizer))##############
        self.model.to(device)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, 6)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
#         print(outputs[0])torch.Size([8, 100, 768])
#         print(outputs[1])torch.Size([8, 768])
#         print(outputs[0][:,0,:])torch.Size([8, 768])
#         print(outputs[1])torch.Size([8, 768])
        x = outputs[1]
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [23]:
from transformers import AutoTokenizer, AutoModel
added_token=['##char##']
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese",additional_special_tokens=added_token)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
import torch
cuda_num = str(6)
device_s = "cuda:" + cuda_num
device = torch.device(device_s if torch.cuda.is_available() else "cpu")
cls=torch.load("../data/cls"+str(ban)+".model",map_location=device)

In [41]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

def get_output(text):
    with torch.no_grad():
        text2id = tokenizer(
            text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
        )
    #     print(text,text2id)
        input_ids=text2id["input_ids"].to(device)
        mask=text2id["attention_mask"].to(device)
    #         print(text2id)
        output = cls(input_ids, attention_mask=mask)
        
        output1=[]
    
        for i in output:#18
            s=''

            for j in i:
                s+=str(j.item())
                s+=','
            s=s[:-1]
            output1.append(s)
            
    return output1


In [42]:
text = ['##char##与n3：啊？','##char##与n3：啊？']
# 爱、乐、惊、怒、恐、哀
print(get_output(text))

['-0.03936658799648285,-0.014446040615439415,0.4240415692329407,0.06290137767791748,-0.021756447851657867,0.005224205553531647', '-0.03936658799648285,-0.014446040615439415,0.4240415692329407,0.06290137767791748,-0.021756447851657867,0.005224205553531647']


In [35]:
train_x=[i.replace('*','##char##') for i in train_x]

In [36]:
result=[]

i=0
batch_size=16
while(i<len(train_x)):
    if i+batch_size<=len(train_x):
        input_x=train_x[i:i+batch_size]
        print('{}:{},len={}'.format(i,i+batch_size,batch_size),end = "\r")
    else:
        input_x=train_x[i:len(train_x)]
        print('{}:{},len={}'.format(i,len(train_x)-1,len(train_x)-i),end = "\r")
        
    pre=get_output(input_x)
#     print(pre)
    result+=pre
#     print(len(result))
    i+=batch_size


In [37]:
end_list=read_list('../data/end_list.txt')
print(end_list[0])
print(len(result))
print(len(end_list))
print(result[0])

34170_0002_A_12
21376
21376
-0.03576308861374855,0.01957029104232788,0.009344331920146942,-0.00970818754285574,0.11695418506860733,0.1636660099029541


In [38]:
ff = open('../data/result'+str(ban)+'.txt', encoding='utf-8', mode='w')
ff.write('id\temotion\n')
for i in range(len(end_list)):
    ff.write(end_list[i]+'\t'+result[i])
    ff.write("\n")
